In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
raw_csv_data = pd.read_csv('Index2018.csv')

In [3]:
df_comp = raw_csv_data.copy()

In [4]:
df_comp.date.describe()
# All the data values are 1. Any single data holds the top value. 
# This is why we need to convert the dates into a datetime type

count           6269
unique          6269
top       02/08/2001
freq               1
Name: date, dtype: object

In [5]:
df_comp.date = pd.to_datetime(df_comp.date, dayfirst=True) # assumes we are plugging in a string in a mm/dd/yyyy form

In [6]:
df_comp.date.describe()

count                    6269
unique                   6269
top       1999-08-23 00:00:00
freq                        1
first     1994-01-07 00:00:00
last      2018-01-29 00:00:00
Name: date, dtype: object

### Setting the index

In [7]:
# setting date column as index
df_comp.set_index('date', inplace=True)

In [8]:
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


In [9]:
df_comp.date.describe()

AttributeError: 'DataFrame' object has no attribute 'date'

### Setting the frequency

h - hourly

w - weekly

d - daily

m - monthly

a - annual

b - business days (excludes weekends)

In [12]:

df_comp = df_comp.asfreq('b')

In [13]:
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


### Handling missing values

.isna() - True/False

In [16]:
df_comp.isna().sum() # setting the frequency to bussiness days generated 8 dates without data available

spx       8
dax       8
ftse      8
nikkei    8
dtype: int64

### Fillna method

- front filling - assigns the value of the previous period

- back filling - assigns the value for the next period

- assigning the same value using average value within the time-series

In [17]:
df_comp.spx = df_comp.spx.fillna(method='ffill')

In [18]:
df_comp.isna().sum()

spx       0
dax       8
ftse      8
nikkei    8
dtype: int64

In [19]:
df_comp.ftse = df_comp.ftse.fillna(method='bfill')

In [20]:
df_comp.dax = df_comp.dax.fillna(value=df_comp.dax.mean())

In [21]:
df_comp.isna().sum()

spx       0
dax       0
ftse      0
nikkei    8
dtype: int64

In [22]:
df_comp.nikkei = df_comp.nikkei.fillna(method='ffill')

### Adding and removing columns

In [23]:
# adding column
df_comp['market_value'] = df_comp.spx

In [24]:
df_comp.describe()

,spx,dax,ftse,nikkei,market_value
count,6277.000000,6277.000000,6277.000000,6277.000000,6277.000000
mean,1288.642547,6080.063363,5423.690398,14597.672753,1288.642547
std,487.868210,2752.604984,1145.568370,4043.795272,487.868210
min,438.920000,1911.700000,2876.600000,7054.980000,438.920000
25%,992.715221,4070.460000,4487.880000,10701.130000,992.715221
50%,1233.761241,5774.380000,5663.300000,15030.510000,1233.761241
75%,1460.250000,7442.660000,6304.630175,17860.470000,1460.250000
max,2872.867839,13559.600000,7778.637689,24124.150000,2872.867839


In [25]:
# deleting column
del df_comp['spx']

In [26]:
df_comp.describe()

,dax,ftse,nikkei,market_value
count,6277.000000,6277.000000,6277.000000,6277.000000
mean,6080.063363,5423.690398,14597.672753,1288.642547
std,2752.604984,1145.568370,4043.795272,487.868210
min,1911.700000,2876.600000,7054.980000,438.920000
25%,4070.460000,4487.880000,10701.130000,992.715221
50%,5774.380000,5663.300000,15030.510000,1233.761241
75%,7442.660000,6304.630175,17860.470000,1460.250000
max,13559.600000,7778.637689,24124.150000,2872.867839


In [27]:
# deleting multiple columns
del df_comp['dax'], df_comp['ftse'], df_comp['nikkei']

In [28]:
df_comp.describe()

,market_value
count,6277.000000
mean,1288.642547
std,487.868210
min,438.920000
25%,992.715221
50%,1233.761241
75%,1460.250000
max,2872.867839


### Splitting data

Splitting into training set and testing set for machine learning

80% - 20% is reasonable

In [29]:
size = int(len(df_comp) * 0.8)
size

5021

In [30]:
# training set
df = df_comp.iloc[:size]
# testing set
df_test = df_comp.iloc[size:]

In [31]:
df.tail()

,market_value
date,
2013-04-01,1562.173837
2013-04-02,1570.252238
2013-04-03,1553.686978
2013-04-04,1559.979316
2013-04-05,1553.278930


In [32]:
df_test.head()

,market_value
date,
2013-04-08,1563.071269
2013-04-09,1568.607909
2013-04-10,1587.731827
2013-04-11,1593.369863
2013-04-12,1588.854623


## Updating set
### using yfinance library

In [33]:
import yfinance
import warnings
warnings.filterwarnings('ignore')

In [34]:
raw_data = yfinance.download(tickers='^GSPC ^FTSE ^N225 ^GDAXI', start='1994-01-07', end='2021-10-10', interval='1d', group_by='ticker', auto_adjust='True', threads=True)

[*********************100%***********************]  4 of 4 completed


In [35]:
df_comp = raw_data.copy()

In [36]:
df_comp['spx'] = df_comp['^GSPC'].Close
df_comp['dax'] = df_comp['^GDAXI'].Close
df_comp['ftse'] = df_comp['^FTSE'].Close
df_comp['nikkei'] = df_comp['^N225'].Close

In [37]:
df_comp = df_comp.iloc[1:] # removing the first elements since we always start 1 period before the first due to time difference
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method='ffill')

In [39]:
size = int(len(df_comp) * 0.8)
size

5792

In [40]:
df = df_comp.iloc[:size]
df_test = df_comp.iloc[size:]